# Mais sobre `SELECT `: agrupamento, subqueries

## Introdução

Vamos continuar nossa investigação sobre o comando `SELECT`. Desta vez usaremos a base de dados 'sakila', uma famosa base de dados de teste construída pelos autores do MySQL. 

Abra a URL [`https://dev.mysql.com/doc/sakila/en/`](https://dev.mysql.com/doc/sakila/en/) para conhecer melhor esta base de dados, que modela uma locadora de DVDs (riam, vocês também serão velhos um dia!). Se você não possui esta base, siga as instruções de instalação na página.

Vamos também construir nosso objeto auxiliar para conectar com a base de dados, como fizemos na última aula.

In [1]:
import mysql.connector
from functools import partial

def get_connection_helper():
    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            print('Executando query:')
            cursor.execute(query, args)
            for result in cursor:
                print(result)

    connection = mysql.connector.connect(
        host='localhost',
        user='megadados',
        password='megadados2020',
        database='sakila'
    )
    return connection, partial(run_db_query, connection)

connection, db = get_connection_helper()

## Aquecimento

Quais os nomes das categorias de filme?

In [2]:
db('''
SELECT
    name
FROM
    category
''')

Executando query:
('Action',)
('Animation',)
('Children',)
('Classics',)
('Comedy',)
('Documentary',)
('Drama',)
('Family',)
('Foreign',)
('Games',)
('Horror',)
('Music',)
('New',)
('Sci-Fi',)
('Sports',)
('Travel',)


Quais atores tem as iniciais "J.D."?

In [3]:
db('''
SELECT
    first_name, last_name
FROM
    actor
WHERE
    first_name like 'J%'
    AND last_name like 'D%'
''')

Executando query:
('JENNIFER', 'DAVIS')
('JUDY', 'DEAN')
('JODIE', 'DEGENERES')
('JULIANNE', 'DENCH')


Liste as cidades brasileiras presentes na base de dados.

In [4]:
db('''
SELECT
    city.city
FROM
    country
    INNER JOIN city USING (country_id)
WHERE
    country = 'Brazil'
''')

Executando query:
('Alvorada',)
('Angra dos Reis',)
('Anpolis',)
('Aparecida de Goinia',)
('Araatuba',)
('Bag',)
('Belm',)
('Blumenau',)
('Boa Vista',)
('Braslia',)
('Goinia',)
('Guaruj',)
('guas Lindas de Gois',)
('Ibirit',)
('Juazeiro do Norte',)
('Juiz de Fora',)
('Luzinia',)
('Maring',)
('Po',)
('Poos de Caldas',)
('Rio Claro',)
('Santa Brbara dOeste',)
('Santo Andr',)
('So Bernardo do Campo',)
('So Leopoldo',)
('Sorocaba',)
('Vila Velha',)
('Vitria de Santo Anto',)


*Quantas* cidades brasileiras tem na base de dados?

In [5]:
db('''
SELECT
    COUNT(city)
FROM
    country
    INNER JOIN city USING (country_id)
WHERE
    country = 'Brazil'
''')

Executando query:
(28,)


Liste os filmes do ator (fictício) "Dan Harris"

In [6]:
db('''
SELECT
    film.title
FROM
    actor 
    INNER JOIN film_actor USING (actor_id)
    INNER JOIN film USING (film_id)
WHERE
    actor.first_name = 'Dan'
    AND actor.last_name = 'Harris'
''')

Executando query:
('BEDAZZLED MARRIED',)
('BOONDOCK BALLROOM',)
('DESTINY SATURDAY',)
('DIVINE RESURRECTION',)
('EYES DRIVING',)
('FELLOWSHIP AUTUMN',)
('GHOST GROUNDHOG',)
('GROOVE FICTION',)
('HILLS NEIGHBORS',)
('HOLIDAY GAMES',)
('INDEPENDENCE HOTEL',)
('INSIDER ARIZONA',)
('JADE BUNCH',)
('LIES TREATMENT',)
('MONTEREY LABYRINTH',)
('REUNION WITCHES',)
('RUN PACIFIC',)
('SCHOOL JACKET',)
('SEVEN SWARM',)
('SIEGE MADRE',)
('STEERS ARMAGEDDON',)
('STRAIGHT HOURS',)
('SUMMER SCARFACE',)
('SUPERFLY TRIP',)
('TITANIC BOONDOCK',)
('TITANS JERK',)
('VANISHING ROCKY',)
('WATERSHIP FRONTIER',)


Quais filmes estão alugados por Florence Woods?

In [7]:
db('''
SELECT
    film.title, customer.first_name, customer.last_name
FROM
    customer 
    INNER JOIN rental USING (customer_id)
    INNER JOIN inventory USING (inventory_id)
    INNER JOIN film USING (film_id)
WHERE
    customer.first_name = 'Florence'
    AND customer.last_name = 'Woods'
    AND rental.return_date IS NULL
''')

# última linha mostra os alugados atualmente/não devolvidos

Executando query:
('CLUB GRAFFITI', 'FLORENCE', 'WOODS')
('BLADE POLISH', 'FLORENCE', 'WOODS')


Para quais línguas não tem nenhum filme na locadora? (Dica: use `LEFT OUTER JOIN`.)

In [9]:
db('''
SELECT
    language.name
FROM
    language
    LEFT OUTER JOIN film USING (language_id)
WHERE 
    film_id IS NULL
''')

Executando query:
('Italian',)
('Japanese',)
('Mandarin',)
('French',)
('German',)


## `DISTINCT`

As vezes desejamos consultar quais os valores distintos de uma coluna. Para isso usamos o qualificador `DISTINCT`. 

Por exemplo: Quais os anos de lançamento dos filmes da base? 

In [10]:
# Vai aparecer muitas cópias de '(2006,)'
db('SELECT release_year FROM film LIMIT 5')

Executando query:
(2006,)
(2006,)
(2006,)
(2006,)
(2006,)


Ops, parece que não tem muita variedade nesta base! Usando `DISTINCT` podemos limpar esse resultado:

In [11]:
db('SELECT DISTINCT release_year FROM film')

Executando query:
(2006,)


### Praticando

Quais clientes estão alugando um DVD agora?

In [12]:
db('''
SELECT
    DISTINCT customer.first_name, customer.last_name
FROM
    customer 
    INNER JOIN rental USING (customer_id)
    INNER JOIN inventory USING (inventory_id)
    INNER JOIN film USING (film_id)
WHERE
    rental.return_date IS NULL
''')

Executando query:
('GAIL', 'KNIGHT')
('GREGORY', 'MAULDIN')
('LOUISE', 'JENKINS')
('WILLIE', 'HOWELL')
('EMILY', 'DIAZ')
('LAURIE', 'LAWRENCE')
('LISA', 'ANDERSON')
('FREDDIE', 'DUGGAN')
('HEATHER', 'MORRIS')
('ROLAND', 'SOUTH')
('NATALIE', 'MEYER')
('SCOTT', 'SHELLEY')
('LOUIS', 'LEONE')
('CATHY', 'SPENCER')
('GUY', 'BROWNLEE')
('JUSTIN', 'NGO')
('JERRY', 'JORDON')
('ALICIA', 'MILLS')
('JEANNE', 'LAWSON')
('DUSTIN', 'GILLETTE')
('MORRIS', 'MCCARTER')
('JUDY', 'GRAY')
('JULIE', 'SANCHEZ')
('RAMONA', 'HALE')
('SONIA', 'GREGORY')
('MARGIE', 'WADE')
('ZACHARY', 'HITE')
('JORDAN', 'ARCHULETA')
('ANDY', 'VANHORN')
('ELMER', 'NOE')
('PEGGY', 'MYERS')
('SERGIO', 'STANFIELD')
('LAURA', 'RODRIGUEZ')
('ANNA', 'HILL')
('MARILYN', 'ROSS')
('MILDRED', 'BAILEY')
('GRACE', 'ELLIS')
('BARRY', 'LOVELACE')
('EDWARD', 'BAUGH')
('CASSANDRA', 'WALTERS')
('BILL', 'GAVIN')
('CURTIS', 'IRBY')
('ALLISON', 'STANLEY')
('COURTNEY', 'DAY')
('GILBERT', 'SLEDGE')
('IVAN', 'CROMWELL')
('WANDA', 'PATTERSON')
('VIOLA',

## Agrupamento

Uma das características mais valiosas de banco de dados é o *agrupamento*. Podemos agrupar os resultados de uma query indicando uma coluna cujos valores serão usados para agrupar os dados.

Por exemplo, considere a seguinte tabela, que chamaremos de 'vendas':

| id | id_item | item | preco |
|--|--|--|--|
| 1 | 1 | A | 5 |
| 2 | 2 | B | 6 |
| 3 | 1 | A | 3 |
| 4 | 3 | C | 7 |
| 5 | 3 | C | 5 |
| 6 | 1 | A | 2 |

Se agruparmos pela coluna 'id_item' teremos 3 conjuntos de resultados:

id_item = 1:

| id | id_item | item | preco |
|--|--|--|--|
| 1 | 1 | A | 5 |
| 3 | 1 | A | 3 |
| 6 | 1 | A | 2 |

id_item = 2:

| id | id_item | item | preco |
|--|--|--|--|
| 2 | 2 | B | 6 |

id_item = 3:

| id | id_item | item | preco |
|--|--|--|--|
| 4 | 3 | C | 7 |
| 5 | 3 | C | 5 |

É como se tivessemos uma lista de tabelas! Isso não é permitido em SQL. Temos que 'resumir' a informação de cada uma das tabelas a uma linha só, o que significa que, para cada coluna, devemos escolher uma dessas opções:
- Resumir a informação da coluna usando uma função de grupo. Podemos somar, tirar a média, contar itens, concatená-los em uma única string, entre outras;
- Para colunas que se relacionam 1 para 1 com a coluna de agrupamento (como a coluna de agrupamento em si, ou a coluna 'item' neste exemplo), manter este valor. Isso acontece frequentemente quando fazemos `JOIN`.
- Não incluir a coluna, caso contrário.

Neste exemplo, podemos tomar a seguinte decisão para cada coluna:
- id: descartar
- id_item: manter valor
- item: manter valor
- preco: vamos calcular a soma dos valores, e renomear esta informação para 'total'

Com isso, obtemos a seguinte tabela:

| id_item | item | total |
|--|--|--|
| 1 | A | 10 |
| 2 | B | 6 |
| 3 | C | 12 |

Por fim, se não queremos id_item, ficamos com a seguinte tabela:

| item | total |
|--|--|
| A | 10 |
| B | 6 |
| C | 12 |

Para obter essa tabela podemos usar o seguinte comando SQL:

```SQL
SELECT 
    item, SUM(preco) as total 
FROM 
    vendas
GROUP BY
    id_item
```

Consulte o capítulo 9 do seu livro texto para conhecer mais sobre agrupamentos.

### Praticando

Quais os 10 atores que mais apareceram em filmes?

In [13]:
db('''
SELECT
    COUNT(actor_id) as film_count, actor.first_name, actor.last_name
FROM
    actor 
    INNER JOIN film_actor USING (actor_id)
    INNER JOIN film USING (film_id)
GROUP BY
    actor_id
ORDER BY
    film_count DESC
LIMIT 10
''')

Executando query:
(42, 'GINA', 'DEGENERES')
(41, 'WALTER', 'TORN')
(40, 'MARY', 'KEITEL')
(39, 'MATTHEW', 'CARREY')
(37, 'SANDRA', 'KILMER')
(36, 'SCARLETT', 'DAMON')
(35, 'VAL', 'BOLGER')
(35, 'VIVIEN', 'BASINGER')
(35, 'GROUCHO', 'DUNST')
(35, 'UMA', 'WOOD')


## Pipeline do comando `SELECT`

Uma versão mais completa do `SELECT` (mas não inteiramente completa - consulte o manual do MySQL) é vista abaixo:

```
SELECT [DISTINCT] <select_header> 
FROM <source_tables>
WHERE <filter_expression>
GROUP BY <grouping_expressions>
HAVING <filter_expression>
ORDER BY <ordering_expressions>
LIMIT <count> 
OFFSET <count>
```

Você já deve ter percebido que o comando `SELECT` tem uma sequência própria de avaliação. Por exemplo, para saber quais filmes custam mais que 3 dinheiros, podemos escrever:

In [14]:
db('''
SELECT 
    COUNT(f.rental_rate)
FROM
    film f
WHERE
    f.rental_rate > 3
''')

Executando query:
(336,)


Observe que o 'apelido' f para a tabela 'film' é definido na cláusula `FROM`, mas usado em `SELECT` e também em `WHERE`.

A ordem de execução do comando `SELECT` é aproximadamente como segue:

1. `FROM <source_tables>`: indica as tabelas que serão usadas nesta query e, conceitualmente, combina estas tabelas através de *produto cartesiano* em uma grande tabela. (Note o termo "*conceitualmente*" que usei: em termos de implementação da query este produto cartesiano raramente é construído.)

2. `WHERE <filter_expression>`: filtra linhas.

3. `GROUP BY <grouping_expressions>`: agrupa conjuntos de linhas.

4. `SELECT <select_heading>`: escolha de colunas e de agregados.

5. `HAVING <filter_expression>`: outra filtragem, esta aplicada apenas **depois** da agregação. Pode usar resultados do processo de agregação. Obriga o uso de `GROUP BY`.

6. `DISTINCT`: Elimina linhas duplicadas.

7. `ORDER BY`: ordena as linhas do resultado.

8. `OFFSET <count>`: Pula linhas do resultado. Requer LIMIT.

9. `LIMIT <count>`: Mantém apenas um número máximo de linhas.

Esta sequencia também serve como dica de como projetar uma query! 
- Comece identificando as tabelas que você deseja usar
- Monte o filtro de linhas, incluindo critérios de `JOIN`
- Agrupe
- Selecione colunas e aplique funções de agregação, conforme necessário
- Filtre com `HAVING`, agora que temos agregação
- O resto é mais fácil, aplique conforme requerido

## `WHERE` versus `HAVING`

Conforme visto acima, temos a cláusula `HAVING` para fazer filtragens *APÓS* agregação. Para que serve isso? Por exemplo, suponha que queremos saber quais atores não compartilham seu sobrenome com nenhum outro ator. Podemos usar a query a seguir:

In [15]:
db('''
SELECT 
    a.first_name, a.last_name
FROM 
    actor a 
GROUP BY 
    a.last_name
HAVING
    COUNT(a.first_name) = 1
ORDER BY
    a.last_name, a.first_name
''')

Executando query:
('ANGELINA', 'ASTAIRE')
('RUSSELL', 'BACALL')
('HARRISON', 'BALE')
('RENEE', 'BALL')
('JULIA', 'BARRYMORE')
('VIVIEN', 'BASINGER')
('VIVIEN', 'BERGEN')
('LIZA', 'BERGMAN')
('CUBA', 'BIRCH')
('KEVIN', 'BLOOM')
('CHRIS', 'BRIDGES')
('LAURENCE', 'BULLOCK')
('MATTHEW', 'CARREY')
('GREG', 'CHAPLIN')
('RUSSELL', 'CLOSE')
('FRED', 'COSTNER')
('SIDNEY', 'CROWE')
('JUDE', 'CRUISE')
('RALPH', 'CRUZ')
('SCARLETT', 'DAMON')
('FRANCES', 'DAY-LEWIS')
('SYLVESTER', 'DERN')
('ALAN', 'DREYFUSS')
('GROUCHO', 'DUNST')
('CHRISTIAN', 'GABLE')
('MERYL', 'GIBSON')
('PARKER', 'GOLDBERG')
('ADAM', 'GRANT')
('MEG', 'HAWKE')
('GEOFFREY', 'HESTON')
('HARVEY', 'HOPE')
('ANGELA', 'HUDSON')
('CARMEN', 'HUNT')
('WHOOPI', 'HURT')
('WOODY', 'JOLIE')
('KIRK', 'JOVOVICH')
('MATTHEW', 'LEIGH')
('JOHNNY', 'LOLLOBRIGIDA')
('GRETA', 'MALDEN')
('ED', 'MANSFIELD')
('ELVIS', 'MARX')
('MORGAN', 'MCDORMAND')
('TOM', 'MIRANDA')
('BETTE', 'NICHOLSON')
('KENNETH', 'PESCI')
('OLYMPIA', 'PFEIFFER')
('GARY', 'PHOENIX'

## Praticando

Liste a duração média dos filmes na categoria 'Drama'.

In [16]:
db("""
SELECT
    AVG(film.length)
FROM
    film 
    INNER JOIN film_category USING (film_id)
    INNER JOIN category USING (category_id)
GROUP BY
    category.name
HAVING
    category.name = "Drama"
""")

Executando query:
(Decimal('120.8387'),)


Liste o nome da categoria e a duração média dos filmes por categoria.

In [17]:
db("""
SELECT
    category.name, AVG(film.length)
FROM
    film 
    INNER JOIN film_category USING (film_id)
    INNER JOIN category USING (category_id)
GROUP BY
    category_id
""")

Executando query:
('Action', Decimal('111.6094'))
('Animation', Decimal('111.0152'))
('Children', Decimal('109.8000'))
('Classics', Decimal('111.6667'))
('Comedy', Decimal('115.8276'))
('Documentary', Decimal('108.7500'))
('Drama', Decimal('120.8387'))
('Family', Decimal('114.7826'))
('Foreign', Decimal('121.6986'))
('Games', Decimal('127.8361'))
('Horror', Decimal('112.4821'))
('Music', Decimal('113.6471'))
('New', Decimal('111.1270'))
('Sci-Fi', Decimal('108.1967'))
('Sports', Decimal('128.2027'))
('Travel', Decimal('113.3158'))


Liste o nome da categoria e a duração média dos filmes por categoria, apenas para categorias cuja duração média de filme excede 120 minutos.

In [17]:
db("""
SELECT
    category.name, AVG(film.length) as avg_len
FROM
    film 
    INNER JOIN film_category USING (film_id)
    INNER JOIN category USING (category_id)
GROUP BY
    category.name
HAVING
    avg_len > 120
""")

Executando query:
('Drama', Decimal('120.8387'))
('Foreign', Decimal('121.6986'))
('Games', Decimal('127.8361'))
('Sports', Decimal('128.2027'))


# Conclusão

Esta aula de hoje foi bastante densa! Dicas de estudo:

- Pratique no seu livro-texto, capítulo 9. Lembre-se que a base 'música' pode ser usada para praticar os comandos SQL vistos no livro.
- Tente criar queries que sirvam de exemplo para os conceitos do livro! A tarefa de criar exemplos é muito instrutiva!

**Leitura prévia**:
Para a próxima aula vamos continuar praticando, com os assuntos dos capítulos 10 e 11, prepare-se para a aula, ok?

Até a próxima!

In [18]:
connection.close()